# **Chapter 5 오차역전파법**

5.1~5.3은 교재 참고

## **5.4 단순한 계층 구현하기**

### **5.4.1 곱셈 계층**



In [0]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self,x,y):
    self.x = x
    self.y = y
    out = x*y

    return out

  def backward(self,dout):
    dx = dout*self.y # x와 y를 바꾼다.
    dy = dout*self.x

    return dx, dy

 곱셈 계층을 구현해보았다.

forward()는 순전파, backward()는 역전파를 처리한다. 

__init__()에서는 인스턴스 변수인 x와 y를 초기화한다. 이 두 변수는 순전파 시의 입력 값을 유지하기 위해 사용한다. 

forward()에서는 x와 y를 인수로 받아 두 값을 곱하여 반환한다. 반면 backward()에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘린다. 

___

인스턴스 변수는 클래스 변수와 마찬가지로 클래스 내에 선언한다. 클래스 변수와의 차이점은 인스턴스에 종속되어 인스턴스 생성시 마다 새로운 저장공간을 할당한다는 점이다. 즉 저장공간이 공유되지 않는다.

In [3]:
# 교재 p.161 그림 5-16의 순전파 구현

apple = 100 # 사과 가격
apple_num = 2 # 사과의 개수
tax = 1.1 # 소비세

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


In [4]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


backward()에서 각 변수에 대한 미분을 구했다. backward()가 받는 인수는 순전파의 출력에 대한 미분이다. 

코드를 실행한 결과는 그림 5-16의 결과와 일치한다.

### **5.4.2 덧셈 계층**

In [0]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self,dout):
    dx = dout*1
    dy = dout*1
    return dx,dy

덧셈 계층을 구현했다. 덧셈 계층에서는 초기화가 필요 없어서 __init__()에서 아무 일도 하지 않았다. (pass는 아무것도 하지 말라는 명령임) 

덧셈 계층의 forward()에서는 입력받은 두 인수 x,y를 더해서 반환한다. 

backward()에서는 상류에서 내려온 미분을 그대로 하류로 흘린다. 

In [0]:
# 교재 pg.163의 그림 5-17 구현

apple = 100 # 사과 가격
apple_num = 2 # 사과 개수
orange = 150 # 오렌지 가격
orange_num = 3 # 오렌지 개수
tax = 1.1 # 소비세

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price,orange_price) #(3)
price = mul_tax_layer.forward(all_price,tax) #(4)

# 역전파
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price,dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange,dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

필요한 계층을 만들어 순전파 메서드인 forward()를 적절한 순서로 호출했다. 그런 다음 순전파와 반대 순서로 역전파 메서드인 backward()를 호출해 원하는 미분이 나오게 했다. 

In [13]:
print(price)

715.0000000000001


In [14]:
print(dapple_num, dapple, dorange, dorange_num, dtax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


코드를 실행한 결과가 그림 5-17의 결과와 일치한다.

## **5.5 활성화 함수 계층 구현하기**

### **5.5.1 ReLU 계층**

ReLU는 x가 0을 초과하면 x를, 0 이하이면 0을 출력하는 함수이다. 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘리고, 순전파 때 x가 0 이하면 역전파 때는 하류로 신호를 보내지 않는다. (0을 보낸다.)

In [0]:
class Relu:
  def __init__(self):
    self.mask = None
  
  def forward(self,x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

ReLU 계층을 구현했다. 

ReLU 클래스는 mask라는 인스턴스 변수를 가진다. mask는 True/False로 구성된 넘파이 배열로, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지한다. 

In [18]:
import numpy as np
x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [19]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


mask 변수가 True/False로 구성된 넘파이 배열을 유지하는 것을 알 수 있다. 

ReLU 계층의 계산 그래프에서 순전파 때의 입력 값이 0 이하면 역전파 때의 값은 0이 되어야 하기 때문에 역전파 때는 순전파 때 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서 전파된 dout을 0으로 설정한다. 

### **5.5.2 Sigmoid 계층**

시드모이드 계층의 계산 그래프와 각 계산 단계는 교재 p.167-169 참고

In [0]:
class Sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self,x):
    out = 1 / (1+np.exp(-x))
    self.out = out

    return out

  def backward(self,dout):
    dx = dout*(1.0-self.out)*self.out

    return dx

Sigmoid 계층을 구현하였다. 

위의 구현에서는 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용했다. 

## **5.6 Affine/Softmax 계층 구현하기**

### **5.6.1 Affine 계층**

In [0]:
X = np.random.rand(2) #입력
W = np.random.rand(2,3) #가중치
B = np.random.rand(3) #편향

In [23]:
X.shape

(2,)

In [24]:
W.shape

(2, 3)

In [25]:
B.shape

(3,)

In [0]:
Y = np.dot(X,W) + B

In [28]:
Y

array([0.34265424, 0.02930574, 0.38885983])

신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 위와 같이 행렬의 곱을 사용했었다. 

위의 Y를 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름이었다. 또한 행렬의 곱 계산에서는 대응하는 차원의 원소 수를 일치시키는 것이 핵심이었다. 

행렬의 곱과 편향의 합을 계산 그래프로 나타내는 것은 교재 p.171-173 참고

### **5.6.2 배치용 Affine 계층**

지금까지 본 Affine 계층은 입력 데이터로 X 하나만을 고려한 것이었다. 이번 절에서 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보겠다.

교재 p.174의 배치용 Affine 계층의 계산 그래프를 확인해보면 X의 형상이 (N,2)로 변화한 것을 알 수 있다. 편향을 더할 때도 주의해주어야 한다. 순전파 때의 편향 덧셈은 X•W에 대한 편향이 각 데이터에 더해진다. 

In [0]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

In [30]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [31]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

순전파의 편향 덧셈이 각각의 데이터에 더해짐을 확인했다. 그래서 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 한다.

In [32]:
dY = np.array([[1,2,3],[4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [33]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

역전파 값이 편향의 원소에 모이는 것을 코드로 보였다.

In [0]:
class Affine:
  def __init__(self,W,b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self,x):
    self.x = x
    out = np.dot(x,self.W)+self.b

    return out

  def backward(self,dout):
    dx = np.dot(dout,self.W.T)
    self.dW = np.dot(self.x.T,dout)
    self.db = np.sum(dout,axis=0)

    return dx

Affine을 구현했다.

### **Softmax-with-Loss 계층**

설명은 p.176-179 참고

In [0]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None #손실
    self.y = None #softmax의 출력
    self.t = None #정답 레이블(원-핫 벡터)

  def forward(self,x,t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y,self.t)
    return self.loss

  def backward(self,dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y-slef.t) / batch_size

    return dx

Softmax-with-Loss 계층을 구현했다.

## **5.7 오차역전파법 구현하기**

In [1]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
    self.params['b2'] = np.zeros(output_size)
 
    # 계층 생성
    self.layers = OrderedDict() #신경망의 계층을 순서가 있는 딕셔너리에 보관한다.
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
 
    self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
      for layer in self.layers.values():
        x = layer.forward(x)
        
      return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
      y = self.predict(x)
      return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
      y = self.predict(x)
      y = np.argmax(y, axis=1)
      if t.ndim != 1 : t = np.argmax(t, axis=1)
      
      accuracy = np.sum(y == t) / float(x.shape[0])
      return accuracy
    
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
      loss_W = lambda W: self.loss(x, t)
      grads = {}
      grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
      grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
      grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
      grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
      return grads
    
    def gradient(self, x, t):
      # forward
      self.loss(x, t)
 
      # backward
      dout = 1
      dout = self.lastLayer.backward(dout)
 
      layers = list(self.layers.values())
      layers.reverse()
      for layer in layers:
        dout = layer.backward(dout)
 
      # 결과 저장
      grads = {}
      grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
      grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
 
      return grads

ModuleNotFoundError: ignored

오차역전파법을 적용한 신경망을 구현했다.

### **5.7.3 오차역전파법으로 구한 기울기 검증하기**

In [2]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

ModuleNotFoundError: ignored

기울기를 검증하는 코드를 구현했다.

___

기울기를 구하는 방법에는 수치 미분을 써서 구하는 방법과 해석적으로 수식을 풀어 구하는 방법이 있다. 후자인 해석적 방법은 오차역전파법을 이용하여 매개변수가 많아도 효율적으로 계산할 수 있었다.

수치 미분은 느리지만 구협이 쉽고 버그가 숨어 있기 어렵기 때문에 오차역전파법을 정확히 구현했는지 확인하기 위해 필요하다. 

이처럼 두 방식으로 구한 기울기가 일치함을 확인하는 작업을 기울기 확인이라고 한다. 

In [3]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

ModuleNotFoundError: ignored

오차역전파법을 사용한 신경망 학습을 구현해보았다.